<a href="https://colab.research.google.com/github/urieliram/statistical/blob/main/Tarea4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea 4
*Pick one of the examples of the chapter that use the data of the book and replicate it in Python. Then, apply the steps in your own data.*

Los datos utilizados en este cuaderno están disponibles aquí: [datasets](https://drive.google.com/drive/folders/159GnBJQDxTY9oYqPBZzdNghyb4Gd9pDS?usp=sharing) 

In [14]:
import numpy as np
import pandas as pd
import itertools
import warnings
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, LarsCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
import scipy.stats
import statsmodels.api as sm
import statsmodels.tools.eval_measures as bias
import seaborn as sns
warnings.filterwarnings("ignore")

## Regresión líneal en detección de vocales
A continuación repetiremos el ejemplo de clasificación de vocales de la lengua inglesa del [libro](https://link.springer.com/book/10.1007/978-0-387-84858-7) en el que se aplica un modelo de discriminante lineal.

Iniciamos cargando los datos de entrenamiento y almacenamos en `y_train`:

In [21]:
## Procesamos datos de entrenamiento 
df = pd.read_csv('vowel_train.csv')
dfy = df['y']  ## Outcome
dfx = df[['x.1','x.2','x.3','x.4','x.5','x.6','x.7','x.8','x.9','x.10']]  ## Predictors
## Procesamos datos de prueba 
df = pd.read_csv('vowel_test.csv')
dfyt = df['y']  ## Outcome
dfxt = df[['x.1','x.2','x.3','x.4','x.5','x.6','x.7','x.8','x.9','x.10']]  ## Predictors

Aplicaremos un modelo de **discriminante líneal** a los datos.